In [1]:
import pandas as pd

# Create mapping between NPRI and EI

In [ ]:
df_npri_iw = pd.read_excel(r'../data/openIO_IW_EI_concordance.xlsx', sheet_name='NPRI_to_IW21')
df_ei_iw = pd.read_excel(r'../data/openIO_IW_EI_concordance.xlsx', sheet_name='EI_to_IW+')

In [ ]:
# Standardize column names for merging
df_npri_iw.rename(columns={'IMPACT World+ flows': 'iw_name'}, inplace=True)
df_ei_iw.rename(columns={'iw name': 'iw_name'}, inplace=True)

In [ ]:
# Add the 'EI' column to df_npri_iw based on matching iw_name with df_ei_iw
df_npri_iw['EI'] = df_npri_iw['iw_name'].map(
    lambda iw: df_ei_iw[df_ei_iw['iw_name'] == iw]['ecoinvent name'].iloc[0]
    if iw in df_ei_iw['iw_name'].values else 'No match found'
)


In [ ]:
# df_npri_iw.to_excel(r'concordances.xlsx', index=False)

# Import and analyze NPRI data

The National Pollutant Inventory Report (NPRI) can be downloaded from [here](https://www.canada.ca/en/services/environment/pollution-waste-management/national-pollutant-release-inventory.html)

In [16]:
#mapping = df_npri_iw
mapping = pd.read_excel(r'../data/openIO_IW_EI_concordance.xlsx', sheet_name='NPRI_to_EI')

In [17]:
# Need to rename it to merge it to NPRI data
mapping.rename(columns={'OpenIO flows': 'Substance Name (English)'}, inplace=True)

In [19]:
mapping

,Substance Name (English),iw_name,EI
0,Carbon dioxide,"Carbon dioxide, fossil","Carbon dioxide, fossil"
1,Methane,"Methane, fossil",Methane
2,Dinitrogen monoxide,Dinitrogen monoxide,Dinitrogen monoxide
3,CF4,"Methane, tetrafluoro-, CFC-14","Methane, tetrafluoro-, R-14"
4,C2F6,"Ethane, hexafluoro-, HFC-116","Ethane, hexafluoro-, HFC-116"
...,...,...,...
348,Propylene oxide,Propylene oxide,Propylene oxide
349,Thiourea,Thiourea,No match found
350,2-Ethoxyethyl acetate,2-Ethoxyethyl acetate,No match found
351,Azo disperse dyes,NaN,No match found


In [11]:
df_npri = pd.read_excel('../data/Air_emissions/NPRI-INRP_DataDonnées_2023.xlsx', sheet_name='INRP-NPRI 2023', skiprows=3)

In [12]:
df_npri

,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,Province,Postal Code,...,Organic Compound Recovery,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling"
0,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.24600
1,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.20100
2,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59600
3,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.22400
4,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2860.32000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63869,2023,34874,Cenovus Energy Inc.,ABBT0084856 (Tank),Calgary,Woodlands County,,Athabasca--Grande Prairie--Peace River,AB,T2P 0M5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.54124
63870,2023,34874,Cenovus Energy Inc.,ABBT0084856 (Tank),Calgary,Woodlands County,,Athabasca--Grande Prairie--Peace River,AB,T2P 0M5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.40002
63871,2023,34879,Kiwetinohk Energy Corp.,Bigstone 01-18 Pad,Calgary,Greenview No. 16,,Athabasca--Grande Prairie--Peace River,AB,T2T 5N3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.27520
63872,2023,34880,Kiwetinohk Energy Corp.,Bigstone 16-32 Pad,Calgary,Greenview No. 16,,Athabasca--Grande Prairie--Peace River,AB,T2P 0C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78100


In [20]:
# We add the Ecoinvent name based on the mapping 
df_npri = df_npri.merge(mapping[['Substance Name (English)', 'EI']],
              on='Substance Name (English)',
              how='left')

In [21]:
df_npri

,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,Province,Postal Code,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
0,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.24600,Zinc
1,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.20100,"Particulates, < 2.5 um"
2,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59600,No match found
3,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.22400,4-Methyl-2-pentanone
4,2023,1,Alberta-Pacific Forest Industries Inc.,Alberta-Pacific Forest Industries Inc.,County of Athabasca,Athabasca County,,Athabasca--Grande Prairie--Peace River,AB,T0A 0M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2860.32000,"Carbon monoxide, fossil"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63869,2023,34874,Cenovus Energy Inc.,ABBT0084856 (Tank),Calgary,Woodlands County,,Athabasca--Grande Prairie--Peace River,AB,T2P 0M5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.54124,NaN
63870,2023,34874,Cenovus Energy Inc.,ABBT0084856 (Tank),Calgary,Woodlands County,,Athabasca--Grande Prairie--Peace River,AB,T2P 0M5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.40002,NaN
63871,2023,34879,Kiwetinohk Energy Corp.,Bigstone 01-18 Pad,Calgary,Greenview No. 16,,Athabasca--Grande Prairie--Peace River,AB,T2T 5N3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.27520,NaN
63872,2023,34880,Kiwetinohk Energy Corp.,Bigstone 16-32 Pad,Calgary,Greenview No. 16,,Athabasca--Grande Prairie--Peace River,AB,T2P 0C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.78100,NaN


In [33]:
#df_npri['NAICS 4 Sector Name (English)'].value_counts()

In [28]:
df_metal = df_npri[df_npri['NAICS 4 Sector Name (English)'] == 'Metal ore mining'].reset_index()

In [30]:
df_metal

,index,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,Province,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
0,116,2023,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,,Abitibi-Témiscamingue,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.190000,"Carbon monoxide, fossil"
1,117,2023,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,,Abitibi-Témiscamingue,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.890000,No match found
2,118,2023,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,,Abitibi-Témiscamingue,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1264.685750,Manganese
3,119,2023,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,,Abitibi-Témiscamingue,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240622.494400,Lead
4,120,2023,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,,Abitibi-Témiscamingue,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.827000,No match found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,57321,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.509600,NaN
1672,57322,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056200,NaN
1673,57323,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.233200,NaN
1674,57324,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.184400,NaN


In [32]:
percentage_matching = (df_metal['EI'].eq('No match found').mean() * 100)
percentage_matching

np.float64(16.16945107398568)

In [47]:
#df_metal['NAICS 6 Sector Name (English)'].value_counts()

In [35]:
df_nickel = df_metal[df_metal['NAICS 6 Sector Name (English)'] == 'Nickel-copper ore mining'].reset_index()

In [36]:
df_nickel

,level_0,index,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
0,62,1957,2023,1233,Glencore Canada Corporation,Sudbury Operations Mines Mill - Onaping Area,Onaping,Greater Sudbury / Grand Sudbury,Greater Sudbury / Grand Sudbury,Northeast / Nord-est,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.791976,Chromium
1,63,1958,2023,1233,Glencore Canada Corporation,Sudbury Operations Mines Mill - Onaping Area,Onaping,Greater Sudbury / Grand Sudbury,Greater Sudbury / Grand Sudbury,Northeast / Nord-est,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,415301.987000,Lead
2,64,1959,2023,1233,Glencore Canada Corporation,Sudbury Operations Mines Mill - Onaping Area,Onaping,Greater Sudbury / Grand Sudbury,Greater Sudbury / Grand Sudbury,Northeast / Nord-est,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.689982,Vanadium
3,65,1960,2023,1233,Glencore Canada Corporation,Sudbury Operations Mines Mill - Onaping Area,Onaping,Greater Sudbury / Grand Sudbury,Greater Sudbury / Grand Sudbury,Northeast / Nord-est,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.996318,Nitrate
4,66,1961,2023,1233,Glencore Canada Corporation,Sudbury Operations Mines Mill - Onaping Area,Onaping,Greater Sudbury / Grand Sudbury,Greater Sudbury / Grand Sudbury,Northeast / Nord-est,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3372.728288,"Ni, Ni 2.3E+0%, Pt 2.5E-4%, Pd 7.3E-4%, Rh 2.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,1670,57320,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080800,NaN
272,1671,57321,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.509600,NaN
273,1672,57322,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056200,NaN
274,1673,57323,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.233200,NaN


## Analysis per company and facility

In [39]:
df_nickel_company = df_nickel.groupby(['Company Name', 'Facility Name', 'City', 'Latitude', 'Longitude']).size().sort_values(ascending=False)
df_nickel_company

Company Name                            Facility Name                                 City                           Latitude   Longitude 
Vale Newfoundland and Labrador Limited  Voisey's Bay Mine Site                        Happy Valley-Goose Bay         56.337780  -62.094400    28
Glencore                                Mine Raglan                                   Laval                          61.691600  -73.666600    27
Glencore Canada Corporation             Sudbury Operations Mines Mill - Onaping Area  Onaping                        46.674200  -81.342500    23
Vale Canada Limited                     Clarabelle Mill                               Copper Cliff                   46.495000  -81.056100    22
Vale Canada Ltd                         Thompson Operations                           Thompson                       55.712860  -97.851060    20
Glencore Canada Corporation             Nickel Rim South Mine                         Skead/City of Greater Sudbury  46.658000  -80.7972

In [41]:
# Group by 'Company Name' and 'Facility Name' and create separate DataFrames
nickel_groups = df_nickel.groupby(['Company Name', 'Facility Name'])


Company Name                            Facility Name                               
Canadian Royalties inc.                 Projet Nunavik Nickel                           13
FNX Mining Company Inc.                 Levack Mine                                      5
                                        McCreedy West Mine                              17
                                        Victoria Advanced Exploration Project            3
Glencore                                Mine Raglan                                     28
Glencore Canada Corporation             Nickel Rim South Mine                           17
                                        Sudbury Operations Mines Mill - Onaping Area    23
Northern Sun Mining Corp.               Redstone Mine and Mill                           9
Vale Canada Limited                     Clarabelle Mill                                 22
                                        Coleman Mine                                     9
     

In [44]:
# Create a dictionary to store individual DataFrames for each Company-Facility
nickel_groups = df_nickel.groupby(['Company Name', 'Facility Name'])

dfs_by_company_facility = {
    f"{company}_{facility}".replace(" ", "_").replace("/", "-"): group
    for (company, facility), group in nickel_groups
}

dfs_by_company_facility.keys()

dict_keys(['Canadian_Royalties_inc._Projet_Nunavik_Nickel', 'FNX_Mining_Company_Inc._Levack_Mine', 'FNX_Mining_Company_Inc._McCreedy_West_Mine', 'FNX_Mining_Company_Inc._Victoria_Advanced_Exploration_Project', 'Glencore_Mine_Raglan', 'Glencore_Canada_Corporation_Nickel_Rim_South_Mine', 'Glencore_Canada_Corporation_Sudbury_Operations_Mines_Mill_-_Onaping_Area', 'Northern_Sun_Mining_Corp._Redstone_Mine_and_Mill', 'Vale_Canada_Limited_Clarabelle_Mill', 'Vale_Canada_Limited_Coleman_Mine', 'Vale_Canada_Limited_Copper_Cliff_Mine_(North)', 'Vale_Canada_Limited_Copper_Cliff_Mine_(South)', 'Vale_Canada_Limited_Creighton_Mine', 'Vale_Canada_Limited_Ellen_Pit', 'Vale_Canada_Limited_Frood-Stobie_Mine', 'Vale_Canada_Limited_Garson_Mine', 'Vale_Canada_Limited_Gertrude_Pit', 'Vale_Canada_Limited_Totten_Mine', 'Vale_Canada_Ltd_Thompson_Operations', "Vale_Newfoundland_and_Labrador_Limited_Voisey's_Bay_Mine_Site"])

In [45]:
# To access a specific one
dfs_by_company_facility['Canadian_Royalties_inc._Projet_Nunavik_Nickel']

,level_0,index,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
258,1268,30466,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0700,Sulfur dioxide
259,1269,30467,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,525.7100,No match found
260,1270,30468,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.4400,NaN
261,1271,30469,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.1100,"Particulates, < 2.5 um"
262,1272,30470,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1759.3400,No match found
263,1273,30471,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1604.7400,Nitrogen oxides
264,1274,30472,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446.4200,"Carbon monoxide, fossil"
270,1669,57319,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0038,NaN
271,1670,57320,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0808,NaN
272,1671,57321,2023,27059,Canadian Royalties inc.,Projet Nunavik Nickel,Kattiniq,Rivière-Koksoak,,Nord-du-Québec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5096,NaN


# Import production data per facility